An "MLEM" algorithm from XRDUA was used in this paper:

"Impurity precipitation in atomized particles evidenced by nano x-ray diffraction computed tomography"
Anne Bonnin; Jonathan P. Wright; Rémi Tucoulou; Hervé Palancher
Appl. Phys. Lett. 105, 084103 (2014)
https://doi.org/10.1063/1.4894009

This python code implements something similar based on a youtube video (https://www.youtube.com/watch?v=IhETD4nSJec)

There are lots of papers from mathematicians in the literature about MART (multiplicative ART). The conversion of latex algebra back and forth into computer code seems to be a bit of a problem for me (Jon Wright - 17 Nov 2023).

In [ ]:
from ImageD11.sinograms.roi_iradon import iradon, radon # to have projection_shifts
from skimage.transform import iradon_sart
import numpy as np, pylab as pl
import skimage.data

def backproject( sino, theta, projection_shifts = None ):
    """ project the sinogram into the sample """
    return iradon( sino, theta, filter_name=None, projection_shifts = projection_shifts )

def forwardproject( sample, theta, projection_shifts = None ):
    """ project the sample into the experiment (sinogram) """
    return radon( sample, theta, projection_shifts = projection_shifts )

def mlem( sino, theta, 
          startvalue = 1,
          projection_shifts = None,
          pad=0, niter=50, 
          divtol=1e-5, ):
    # 
    # Also called "MART" for Multiplicative ART
    # This keeps a positivity constraint for both the data and reconstruction
    #
    # This implementation was inspired from from:
    # https://www.youtube.com/watch?v=IhETD4nSJec
    # by Andrew Reader
    #
    # ToDo : implement a mask
    # ToDo : check about the corners / circle=False aspects
    #
    # Number of pixels hitting each output in the sample:
    sensitivity_image = backproject( np.ones_like(sino), theta,
                                     projection_shifts = projection_shifts )
    recip_sensitivity_image = 1./sensitivity_image
    # The image reconstruction:
    mlem_rec = np.empty( sensitivity_image.shape, np.float32)
    mlem_rec[:] = startvalue
    for i in range(niter):
        calc_sino = forwardproject( mlem_rec, theta, projection_shifts = projection_shifts )
        ratio = sino / (calc_sino + divtol )
        correction = recip_sensitivity_image * backproject( ratio, theta, 
                                                           projection_shifts = projection_shifts )
        mlem_rec *=  correction
    return mlem_rec

In [ ]:
im = skimage.data.shepp_logan_phantom().astype(np.float32)
pl.imshow(im)

In [ ]:
# generate some angles from a diffraction pattern

import ImageD11.transform, ImageD11.unitcell
import scipy.spatial.transform

# Angles to use are going to correspond to 5 rings of fcc
a = 3.06
cell = ImageD11.unitcell.unitcell( [a,a,a,90,90,90],'F')
cell.makerings(2)
hkls = []
for ds in cell.ringds[:5]:
    hkls += cell.ringhkls[ds]
hkls = np.array( hkls )
hkls.shape

In [ ]:
UB = scipy.spatial.transform.Rotation.random( random_state = 97 ).as_matrix()
gvecs = np.dot( UB/a, hkls.T )

In [ ]:
tth, (eta0, eta1), (omega0, omega1) = ImageD11.transform.uncompute_g_vectors(gvecs, 12.39/50)

In [ ]:
measured = np.concatenate( [ omega0[ np.abs( np.sin( np.radians( eta0 ) ) ) > 0.1 ],
                             omega1[ np.abs( np.sin( np.radians( eta1 ) ) ) > 0.1 ] ] )
print(eta0.shape, measured.shape, measured.min(), measured.max())

In [ ]:
order = np.argsort( measured )
theta = measured[ order ]
theta = theta[ theta > 0 ]

In [ ]:
sino = forwardproject( im, theta )

In [ ]:
pl.imshow(sino, aspect='auto', interpolation='nearest')

In [ ]:
def disp(im, title):
    f, a = pl.subplots(1,2,figsize=(12,5))
    f.colorbar( a[1].imshow(im), ax=a[0] )
    a[1].set(title=title, xticks=(), yticks=())
    a[0].plot( im[im.shape[0]//2] )
    a[0].plot( im[:,im.shape[1]//2] )
    pl.show()

In [ ]:
fbp = iradon( sino, theta, output_size=im.shape[0])
disp(fbp,'Hann filter')

In [ ]:
mlr = mlem( sino, theta, niter=1)
n = 1
disp( mlr, f'mlem {n} steps' )
for niter in (1,4,20,100):
    mlr = mlem( sino, theta, startvalue = mlr, niter=niter)
    n += niter
    disp( mlr, f'mlem {n} steps' )

In [ ]:
rsart = skimage.transform.iradon_sart(sino, theta )
disp( rsart, 'iradon_sart')